In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/Shareddrives/Kaggle/Kaggle/Scripts/')

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import FunctionTransformer

## Store separate dataset into lists

In [4]:
train = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/trainknn_region.csv')
test = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/test_region.csv')

In [6]:
cregion = ['BSh', 'Cfa', 'BSk', 'BWk', 'BWh', 'Csa', 'Csb', 'Cfb', 'Dfb', 'Dsc', 'Dfc', 'Dfa', 'Dsb', 'Dwa', 'Dwb']
cregion_train = []
cregion_test = []
index_test = []

In [7]:
for i in cregion:
  subtrain = train.loc[train['climateregions__climateregion'] == i].drop(['climateregions__climateregion','index'],axis = 1, inplace=False)
  cregion_train.append(subtrain)
  subtest = test.loc[test['climateregions__climateregion'] == i].drop(['climateregions__climateregion'],axis = 1, inplace=False)
  index_test.append(subtest['index'])
  subtest = subtest.drop(['index'],axis = 1, inplace=False)
  cregion_test.append(subtest)

In [8]:
len(cregion_train)

15

In [9]:
len(cregion_test)

15

In [10]:
len(index_test)

15

## Run Model For Each Regional Data

### LightGBM

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
import lightgbm as lgbm
from tqdm import tqdm

In [15]:
target = 'contest-tmp2m-14d__tmp2m'

In [71]:
preds = []
for i in range(0,len(cregion)):
  X_train = cregion_train[i].drop([target], axis=1)
  features = X_train.columns
  X_test = cregion_test[i][features]
  y_train = cregion_train[i][target]
  length = len(cregion_train[i])
  params= {
    'learning_rate':0.05,
    'num_leaves': 16,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    'min_data_in_leaf': int(length/300),
    'max_depth': 8,
    'num_iterations': int(length/300)
  }

  model = lgbm.LGBMRegressor(**params)
  model.fit(X= X_train,
                y= y_train
              )
  preds.append(model.predict(X_test))

In [72]:
for i in range(0,len(cregion)):
  print(len(preds[i]),len(index_test[i]))

427 427
4331 4331
11651 11651
1159 1159
793 793
793 793
3416 3416
366 366
4392 4392
244 244
1037 1037
1891 1891
671 671
122 122
61 61


In [73]:
print(preds[0])

[25.06989315 25.06989315 25.35086458 25.35086458 25.41388227 25.41388227
 25.41388227 25.35086458 25.35086458 24.99431844 24.99431844 24.99431844
 24.99431844 24.99431844 24.99431844 24.99431844 24.99431844 24.99431844
 24.99431844 24.99431844 24.90426089 24.84449493 24.78240464 24.78240464
 24.70528849 24.51162138 24.34748006 24.04187429 23.56979842 23.32965193
 23.32965193 23.32965193 23.32965193 23.32965193 23.32965193 23.32965193
 23.26237844 23.20416269 23.20416269 23.20416269 23.20416269 23.20416269
 23.20416269 23.20416269 23.20416269 23.20416269 23.20416269 23.20416269
 23.20416269 23.20416269 23.0764443  22.68220603 23.00585499 22.44823042
 22.12416904 22.05368196 22.05368196 22.05368196 21.85496492 21.49047184
 20.93809195 23.56674822 23.56674822 23.56674822 23.56674822 23.56674822
 23.93745697 23.93745697 23.93745697 23.93745697 23.93745697 23.8646928
 23.8646928  23.8646928  23.8646928  23.8646928  23.8646928  23.8646928
 23.72353427 23.62759651 23.55048037 23.38633905 23.3

In [74]:
print(index_test[0])

       index
0     375734
1     375735
2     375736
3     375737
4     375738
...      ...
2191  377925
2192  377926
2193  377927
2194  377928
2195  377929

[427 rows x 1 columns]


In [79]:
for i in range(0,len(cregion)):
  preds[i] = pd.DataFrame(preds[i], columns=[target], index=index_test[i].index)
  preds[i]['index'] = index_test[i]

In [80]:
preds[0]

,contest-tmp2m-14d__tmp2m,index
0,25.069893,375734
1,25.069893,375735
2,25.350865,375736
3,25.350865,375737
4,25.413882,375738
...,...,...
2191,19.375802,377925
2192,19.375802,377926
2193,19.236292,377927
2194,19.288982,377928


In [88]:
pred_final = pd.concat(preds,axis=0,ignore_index=False)
print(pred_final)

       contest-tmp2m-14d__tmp2m   index
0                     25.069893  375734
1                     25.069893  375735
2                     25.350865  375736
3                     25.350865  375737
4                     25.413882  375738
...                         ...     ...
26103                  7.301177  401837
26104                  7.485410  401838
26105                  7.485410  401839
26106                  7.485410  401840
26107                  7.485410  401841

[31354 rows x 2 columns]


In [90]:
sorted_data = pred_final.sort_values('index')

In [91]:
sample = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Output/sample_solution.csv')
preds1 = sample.copy()
preds1['contest-tmp2m-14d__tmp2m'] = sorted_data[target]
with open('/content/drive/Shareddrives/Kaggle/Kaggle/Output/solution_v23.csv', 'w', encoding = 'utf-8-sig') as f:
    preds1.to_csv(f,index=False)